In [1]:
from keras.utils import HDF5Matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.utils import to_categorical
import keras
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPool2D, UpSampling2D, Concatenate
from tensorflow_core.compiler.tf2xla.python.xla import concatenate
import src.pcam_loader as data
import numpy as np
import pandas as pd
import imageio
from IPython.display import Image

Using TensorFlow backend.


## Load pcam data

In [2]:
# Load data from /User/Username/.keras/datasets/pcam
dataset = data.load_data()
x_train, y_train, meta_train =  dataset[0]
x_valid, y_valid, meta_valid =  dataset[1]
x_test, y_test, meta_test =  dataset[2]

In [3]:
# y_train = np.array(y_train[:])
# y_train = y_train.reshape(-1, 1)
# y_valid = np.array(y_valid[:])
# y_valid = y_valid.reshape(-1, 1)
# y_test = np.array(y_test[:])
# y_test = y_test.reshape(-1, 1)

In [4]:
input_height = 96
input_width = 96
input_channel = 3 
n_classes = 1
batch_size = 64
epochs = 10

In [5]:
#Input of CNN
img_input = Input(shape=(input_height, input_width, input_channel))

# First layer encoder with 2 Conv and MaxPooling
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = Dropout(0.2)(conv1)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPool2D((2, 2))(conv1)

# Second layer encoder with 2 Conv and MaxPooling
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Dropout(0.2)(conv2)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPool2D((2, 2))(conv2)

# Third layer decoder with 2 Conv and Dropout
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

# Fourth layer decoder with 2 Conv and Dropout
up1 = Concatenate(axis=-1)([UpSampling2D((2, 2))(conv3), conv2])
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

# Fifth layer decoder with 2 Conv and Dropout
up2 = Concatenate(axis=-1)([UpSampling2D((2, 2))(conv4), conv1])
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5)

# Output of classification model
output = Conv2D( n_classes, (1, 1), padding='same')(conv5)
output = Flatten()(output)
output = Dense(n_classes, activation='softmax')(out)



In [6]:
# Compile model 

model = Model(input=[img_input], output=[out])
model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(),metrics=['accuracy'])


C:\Users\matho\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 96, 96, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 96, 96, 32)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 96, 96, 32)   9248        dropout_1[0][0]                  
____________________________________________________________________________________________

### Train the model

In [8]:

model.fit(
    x_train,
    to_categorical(y_train),
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_valid, to_categorical(y_valid)),
    shuffle='batch'
)

ValueError: Error when checking target: expected conv2d_11 to have shape (96, 96, 1) but got array with shape (1, 1, 2)

In [ ]:
model.save("model_autoencoder.h5")

In [ ]:
test_eval = model.evaluate(x_test, to_categorical(y_test), verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

